In [1]:
import tensorflow as tf

In [2]:
import keras
from keras.layers import *

Using TensorFlow backend.


In [3]:
import tensorflow_datasets as tfds

In [4]:
dataset = tfds.load(
    name="higgs",
    split=tfds.Split.TRAIN,
    data_dir="/cluster/scratch/bramva/deep-learning-course/data",
#     in_memory=True,
)
dataset = dataset.shuffle(10, reshuffle_each_iteration=False)

In [24]:
dataset

<ShuffleDataset shapes: {class_label: (), jet_1_b-tag: (), jet_1_eta: (), jet_1_phi: (), jet_1_pt: (), jet_2_b-tag: (), jet_2_eta: (), jet_2_phi: (), jet_2_pt: (), jet_3_b-tag: (), jet_3_eta: (), jet_3_phi: (), jet_3_pt: (), jet_4_b-tag: (), jet_4_eta: (), jet_4_phi: (), jet_4_pt: (), lepton_eta: (), lepton_pT: (), lepton_phi: (), m_bb: (), m_jj: (), m_jjj: (), m_jlv: (), m_lv: (), m_wbb: (), m_wwbb: (), missing_energy_magnitude: (), missing_energy_phi: ()}, types: {class_label: tf.float32, jet_1_b-tag: tf.float64, jet_1_eta: tf.float64, jet_1_phi: tf.float64, jet_1_pt: tf.float64, jet_2_b-tag: tf.float64, jet_2_eta: tf.float64, jet_2_phi: tf.float64, jet_2_pt: tf.float64, jet_3_b-tag: tf.float64, jet_3_eta: tf.float64, jet_3_phi: tf.float64, jet_3_pt: tf.float64, jet_4_b-tag: tf.float64, jet_4_eta: tf.float64, jet_4_phi: tf.float64, jet_4_pt: tf.float64, lepton_eta: tf.float64, lepton_pT: tf.float64, lepton_phi: tf.float64, m_bb: tf.float64, m_jj: tf.float64, m_jjj: tf.float64, m_jlv:

In [5]:
approximate_test_set_fraction = .1
one_in_every = round(1 / approximate_test_set_fraction)

In [6]:
test_dataset = dataset.enumerate() \
    .filter(lambda i, x: i % one_in_every == 0) \
    .map(lambda i, x: x)
train_dataset = dataset.enumerate() \
    .filter(lambda i, x: i % one_in_every != 0) \
    .map(lambda i, x: x)

W1227 01:42:07.452457 47027978444288 ag_logging.py:146] Entity <function <lambda> at 0x2ac713e252f0> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: expected exactly one node node, found []
W1227 01:42:07.499068 47027978444288 ag_logging.py:146] Entity <function <lambda> at 0x2ac713e25510> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: expected exactly one node node, found []
W1227 01:42:07.557889 47027978444288 ag_logging.py:146] Entity <function <lambda> at 0x2ac713e25730> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSIT

In [7]:
features = ['jet_1_b-tag', 'jet_1_eta', 'jet_1_phi', 'jet_1_pt', 'jet_2_b-tag', 'jet_2_eta', 'jet_2_phi', 'jet_2_pt', 'jet_3_b-tag', 'jet_3_eta', 'jet_3_phi', 'jet_3_pt', 'jet_4_b-tag', 'jet_4_eta', 'jet_4_phi', 'jet_4_pt', 'lepton_eta', 'lepton_pT', 'lepton_phi', 'm_bb', 'm_jj', 'm_jjj', 'm_jlv', 'm_lv', 'm_wbb', 'm_wwbb', 'missing_energy_magnitude', 'missing_energy_phi']

In [27]:
model = keras.Sequential([
    InputLayer(input_shape=(28,)),
    BatchNormalization(),
    Dense(30, activation="sigmoid"),
    Dense(50, activation="sigmoid"),
    Dense(50, activation="sigmoid"),
    Dense(25, activation="sigmoid"),
    Dense(1, activation="sigmoid"),
])

In [28]:
model.compile(
    "adam",
    "binary_crossentropy",
    metrics=["accuracy"],
)

In [29]:
batch_size = 10

In [30]:
def generator(dataset):
    while True:
        for item in dataset:
            x = tf.concat(
                [tf.reshape(item[name], (-1, 1)) for name in features],
                axis=1,
            )
            yield x, item["class_label"]

In [33]:
model.fit(
    generator(train_dataset.batch(batch_size)),
    steps_per_epoch=round(2_000_000 / batch_size / 10),
    epochs=1,
    verbose=1,
#     workers=20,
#     use_multiprocessing=False,
)

Epoch 1/1
20000/20000 [==============================] - 95s 5ms/step - loss: 0.6919 - accuracy: 0.5290


In [34]:
model.evaluate(
    generator(test_dataset.batch(1000)),
    steps=100,
)

100/100 [==============================] - 99s 989ms/step


[0.691149115562439, 0.5276200175285339]

In [32]:
model.evaluate(
    generator(test_dataset.batch(1000)),
    steps=100,
)

100/100 [==============================] - 100s 996ms/step


[0.6911624670028687, 0.5276200175285339]